<a href="https://colab.research.google.com/github/Ksenichiro/brave_rats_LLM_benchmarck/blob/main/%D0%86%D0%BD%D1%82%D0%B5%D0%B3%D1%80%D0%B0%D1%86%D1%96%D1%8F_LLM_%D0%B4%D0%BE_%D0%B3%D1%80%D0%B8_BraveRats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install --upgrade google-generativeai

In [ ]:
pip install --upgrade openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.6 MB/s eta 0:00:00


In [ ]:
import random
import openai
from openai import OpenAI

import time
import pandas as pd

import re
import os

In [ ]:

system_prompt = """
  You are a expert strategic tabletop game player.
  Game is called "brave rats".
  Strictly follow the rules:
    - At the beginning of the game You will have eight available cards.
    - Each game round you have to give a card from one that is left available and not played by YOU already.
    - Game will last until 4 wins or no cards left to choose from.
    - During the round you have to choose EXACTLY one card to win a round and optimize a wining strategy.
    - Each card has a STRENGTH and a special power that determine who wins the round.
    - The card with the HIGHEST STRENGTH wins, but some special powers can overrule this.
    - The special powers are mandatory and always count.
    - In the case of a draw (the strength of both cards is equal), the round is put on HOLD. The winner of the next round also wins the previous round that was on HOLD (it may happen that several rounds in a row are on hold) that is dramatically affecting your win rate.

All card given at the start of the game:
  - STRENGTH: 0 CARD: Musician POWER: Wins over all powers, except the 5 (Wizard).
  - STRENGTH: 1 CARD: Princess POWER: If your opponent plays the 7 (Prince), the game ends immediately and you win, regardless of the number of victories. However, the 0 (Musician) still puts the round on hold.
  - STRENGTH: 2 CARD: Spy POWER: Next round, your opponent reveals his card before you choose yours. If both players choose the 2 (Spy) the same round, their powers nullify each other.
  - STRENGTH: 3 CARD: Assassin POWER: The lowest strength wins The lowest strength wins, except against a 7 (Prince).
  - STRENGTH: 4 CARD: Ambassador POWER: If you win with this card, it counts as 2 victories. This effect is valid even if the round is on hold. If the player who played the 5 wins the round following a hold, it’s worth 3 victories.
  - STRENGTH: 5 CARD: Wizard POWER: Nullifies the special power of your opponent’s card. which simply means that the highest card wins the round. Ex: 5 wins 4, 7 wins 0
  - STRENGTH: 6 CARD: General POWER: The card in your next round gets +2 strength. The 6’s power affects the next round. If the opponent plays the 5 (Wizard) in the next round, it does not nullify the 6 - (General) power.
  - STRENGTH: 7 CARD: Prince POWER: You win the round. This card always wins the round except against the 1 (Princess) and the 0 (Musician). It doesn’t matter if the 3 (Assassin) has turned the tables, the 7 (Prince) beats the 3 (Assassin). However, the 0 (Musician) still puts the round on hold. Warning: if the opponent plays the 1 (Princess) against your 7 (Prince), he doesn’t just win the round, he wins the game!

  You will be given current game statistics in the format:
  GAME HISTORY:
  ROUND 1, YOU: STRENGTH - (CARD NAME), OPPONENT: STRENGTH - (CARD NAME). {YOU/OPPONENT} WINS or HOLD

  think step by step how to win the whole game and not a single round.

  ALWAYS REPLY WITH THE card number/STRENGTH, never and under any circumstances don't output any other text except card number that left.

"""
result_matrix = [ #A - apple roun win, H - hold, Y - yarg round win, AW - apple GAME WIN, YW - yarg GAME WIN
    ['G', 'A', 'A', 'A', 'A', 'A', 'YW', 'H'],
    ['Y', 'H', 'A', 'A', 'Y', 'A', 'A', 'H'],
    ['Y', 'Y', 'H', 'A', 'A', 'A', 'A', 'A'],
    ['Y', 'Y', 'Y', 'H', 'Y', 'A', 'A', 'H'],
    ['Y', 'A', 'Y', 'A', 'G', 'Y', 'Y', 'H'],
    ['Y', 'Y', 'Y', 'Y', 'A', 'H', 'A', 'H'],
    ['AW', 'Y', 'Y', 'Y', 'A', 'Y', 'H', 'H'],
    ['H', 'H', 'Y', 'H', 'H', 'H', 'H', 'H']
]

win_strategy_list = [       #win  #lose #tie #non lose rate
   [0, 7, 1, 4, 6, 5, 3],   #[355, 61, 995, 0.9567682494684621]
   [3, 0, 7, 1, 4, 5, 6],   #[717, 87, 1293, 0.9585121602288984]
   [3, 0, 7, 1, 6, 5, 4],   #[538, 60, 1084, 0.9643281807372176]
   [3, 7, 0, 1, 4, 5, 6],   #[518, 79, 1363, 0.9596938775510204]
   [3, 7, 0, 1, 6, 5, 4],   #[419, 55, 1186, 0.9668674698795181]
   [7, 0, 1, 4, 6, 5, 3],   #[402, 56, 1184, 0.9658952496954933]
   [7, 0, 3, 1, 5, 4, 6],   #[723, 93, 1341, 0.9568845618915159]
   [7, 1, 4, 6, 5, 3, 0],   #[44, 51, 1021, 0.9543010752688172]
   [7, 3, 0, 1, 4, 5, 6],   #[555, 77, 1148, 0.9567415730337079]
   [7, 3, 0, 1, 6, 5, 4]
]

win_strategy_list_6 = [
   [7, 0, 1, 6, 5, 3],   #[41, 1, 0, 0.9761904761904762]
   [7, 0, 3, 6, 5, 1],   #[78, 2, 0, 0.975]
   [7, 0, 6, 5, 3, 1],   #[82, 3, 0, 0.9647058823529412]
   [7, 3, 0, 6, 5, 1],   #[63, 2, 0, 0.9692307692307692]
   [7, 6, 0, 5, 3, 1]   #[85, 2, 0, 0.9770114942528736]
]
win_strategy_list_5 = [7, 0, 5, 3, 1] #   [19, 1, 0, 0.95]


In [ ]:
def format_card(card_1):
    card_1=int(card_1)
    card_to_number_mapping= ["musician", "princess", "spy", "assassin", "ambassador", "wizard", "general", "prince"]
    return f"CARD {card_1} - {card_to_number_mapping[card_1]}"


In [ ]:
def format_user_message(player_name, game_history, cards_available, spy_card):

    game_format = ""
    #for turn in turns
    for i, round in enumerate(game_history):
      if i == 0:
        game_format += "GAME HISTORY: \n"
      card_1 = round[0]
      card_2 = round[1]
      round_winner_name = round[2]
      if round_winner_name == player_name:
        winner = "YOU WIN"
      elif round_winner_name == "HOLD":
        winner = "HOLD"
      else:
        winner = "OPPONENT WIN"


      game_format += f"ROUND {i+1}, YOUR's {format_card(card_1)}, OPPONENT's {format_card(card_2)}. {winner}\n"

    if spy_card != '' and spy_card != None:
        game_format += f'On previous round you played a 2 (spy), so now you know your opponents current ROUND card. OPPONENT {format_card(spy_card)}. Choose the best card following the rules to bit your opponent.'
    game_format += "Your ONLY Available cards left:" + ",".join([format_card(card) for card in cards_available])
    game_format += "\nYour turn to give the best strategic card from one of the available. Remember to answer only as card strength/number"
    return game_format

In [ ]:
def call_gemini(user_message, model = "gemini-1.5-flash-latest"):
  import google.generativeai as genai
  genai.configure(api_key="your-api-Key")
  # Create the model
  # See https://ai.google.dev/api/python/google/generativeai/GenerativeModel
  generation_config = {
    "temperature": 1,
    "top_p": 0.95,
    "top_k": 64,
    "max_output_tokens": 10,
    "response_mime_type": "text/plain",
  }
  model = genai.GenerativeModel(
    model_name="gemini-1.5-flash-latest",
    safety_settings=[],
    system_instruction=[system_prompt],
    generation_config=generation_config,
  )

  chat_session = model.start_chat(history=[])

  response = chat_session.send_message(user_message)
  return response.text

In [ ]:
def call_open_ai(user_message, model = "gpt-4-turbo"):
    import openai
    from openai import OpenAI

    client = OpenAI(api_key = "your-api-Key")
    completion = client.chat.completions.create(
      model=model,
      # We need only output as number
      max_tokens=10,
      messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_message}
      ]
    )
    return completion.choices[0].message.content

In [ ]:
class Game:

  def __init__(self, cards_limit = 0):
    self.reset(cards_limit)

  def reset(self, cards_limit = 0):
    self.playerA_score = 0
    self.playerY_score = 0

    self.numbers_availableA = list(range(8))
    self.numbers_availableY = list(range(8))
    self.win_score = 4
    self.tie_range = 8

    if cards_limit > 0:
      self.limit_game(cards_limit)

    self.holdA = 0
    self.holdY = 0

    self.game_over = 0 # 1 - win A, 2 - win Y, 3 - draw
    self.general_bonusA = 0
    self.general_bonusY = 0

    self.assasin_flag = 0

    self.spyA_flag = 0
    self.spyY_flag = 0

    self.turn = 0
    if cards_limit == 1:
      self.win_strategy = random.choice(win_strategy_list)
    if cards_limit == 2:
      self.win_strategy = random.choice(win_strategy_list_6)
    if cards_limit == 3:
      self.win_strategy = win_strategy_list_5
    #print(" Win strategy this game: ", self.win_strategy)
    self.predefined_game = [(0,0), (1, 1), (2, 2), (3,3), (4,4), (5,5), (6,6), (7,7)]

  def limit_game(self, cards_limit):
    if cards_limit > 0:
      self.numbers_availableA.remove(2) #Прибираємо Шпигуна
      self.numbers_availableY.remove(2)
      self.tie_range = 7
    if cards_limit > 1:
      self.numbers_availableA.remove(4) #Прибираємо Амбасадора
      self.numbers_availableY.remove(4)
      self.win_score = 3
      self.tie_range = 6
    if cards_limit > 2:
      self.numbers_availableA.remove(6) #Прибираємо Генерала
      self.numbers_availableY.remove(6)
      self.tie_range = 5
    if cards_limit > 3:
      self.numbers_availableA.remove(5) #Прибираємо Мага
      self.numbers_availableY.remove(5)
      self.win_score = 2
      self.tie_range = 4
    if cards_limit > 4:
      self.numbers_availableA.remove(0) #Прибираємо Музиканта
      self.numbers_availableY.remove(0)
      self.tie_range = 3



  def get_win_strategy(self, player_name, game_history, cards_available, spy_card):
    """ Player automatically selects from the random win list."""
    current_turn = len(game_history)

    choice = self.win_strategy[current_turn]
    print("Win strategy :", self.win_strategy, "this turn played: ", choice)
    return choice

  def get_predefined_player(self, player_name, game_history, cards_available, spy_card):

    """ Player automatically selects from the list."""
    current_turn = len(game_history)
    choice = self.predefined_game[current_turn]
    if player_name == "list_player_1":
        return choice[0]
    elif player_name == "list_player_2":
        return choice[1]

    player_choice = random.choice(cards_available)

    return player_choice

  def get_player_choice(self, player_name, game_history, cards_available, spy_card):

    player_choice = random.choice(cards_available)
    print("Player:" ,player_name, "Numbers available:", cards_available, "Spy card: ", spy_card, " Choice: ", player_choice)
    return player_choice
  #
  def get_gpt_choice(self, player_name, game_history, cards_available, spy_card):
    if len(cards_available) == 1:
      # GPT has no option, last ca
      return cards_available[0]
    user_message = format_user_message(player_name, game_history, cards_available, spy_card)
    print("Player:" ,player_name, "Request:", user_message)
    for _ in range(5):
      ai_result = ""
      try:
        if 'gpt' in player_name:
          ai_result = call_open_ai(user_message, player_name)
        elif 'gemini' in player_name:
          ai_result = call_gemini(user_message, player_name)
        if not ai_result:
          continue

        number_mid = re.search(r'\d+', ai_result)
        number = int(number_mid.group())
        if number not in cards_available:
          continue
        print(player_name, " answer:", number,". ", format_card(number))
        return number
      except Exception as ex:
        if 'Resource has been exhausted' in str(ex) or "Too Many Requests" in str(ex) or "Rate Limit" in str(ex):
            timeout = 25
            print(f"AI resource has been exhausted, sleeping to try again in {timeout} seconds...")
            time.sleep(timeout)
            continue
        print(str(ex))
        continue
    return -1

  def spy_table_choice(self, player_name, game_history, cards_available, opponent_choice):
    # choise to position EXPl choise 7 pos 0, choise 3 pos 4|| 7 - choise
    # Subtract each element in cards_available from 7 to make indexlist of array
    print("Spy choise!")
    print("Player:" ,player_name, "Numbers available:", cards_available, "Spy card: ", opponent_choice)
    if player_name == 'A':
      option_list = [row[7-opponent_choice] for row in result_matrix]
    if player_name == 'Y':
      option_list = result_matrix[7-opponent_choice]

    print("Option list")
    print(option_list)
    adjusted_indices = [7 - x for x in cards_available]
    new_options = [option_list[i] for i in adjusted_indices]

    if player_name == 'Y':
      WIN = 'YW'
      RWIN = 'Y'
    elif player_name == 'A':
      WIN = 'AW'
      RWIN = 'A'
    RHOLD ='H'


    # Find the first occurrence of WIN in option_list
    first_yw_index = next((i for i, x in enumerate(new_options) if x == WIN), None)

    # If WIN is not found, find the first occurrence of RWIN
    if first_yw_index is None:
        first_yw_index = next((i for i, x in enumerate(new_options) if x == RWIN), None)
    # If RWIN is not found, find the first occurrence of RHOLD
    if first_yw_index is None:
        first_yw_index = next((i for i, x in enumerate(new_options) if x == RHOLD), None)
    #FIRST ELEMENT IF NO OPTIONS FOUND
    if first_yw_index is None:
      first_yw_index=0


    resultLetter = new_options[first_yw_index]
    resultNumber = 7-adjusted_indices[first_yw_index]

    print("Spy prediction and Result:"+resultLetter +" "+ str(resultNumber))



    return resultNumber

  def play_round(self, playerA_choice, playerY_choice):

    playerA_power = playerA_choice
    playerY_power = playerY_choice

    #General ruling | Check if there a bonus, add donus if chosen card is general
    if self.general_bonusA != 0:
      playerA_power += 2
      self.general_bonusA = 0
      #print("Player A gain general bonus! A choice:"+ str(playerA_choice) + " A power: " + str(playerA_power))

    if self.general_bonusY != 0:
      playerY_power += 2
      self.general_bonusY = 0
      #print("Player y gain general bonus! y choice:"+ str(playerY_choice) + " y power: " + str(playerY_power))

    if playerA_choice == 6:
      self.general_bonusA = 2
      #print("Player A gain general bonus!")

    if playerY_choice == 6:
      self.general_bonusY = 2
      #print("Player y gain general bonus!")

    #Assasin Rule
    if playerA_choice == 3 or playerY_choice == 3:
      self.assasin_flag = 1
    #Ambassador Rule
    if playerA_choice == 4:
      self.holdA +=1
      #print("Extra win point for A")
    if playerY_choice == 4:
      self.holdY +=1
      #print("Extra win point for Y")
    #Spy Rule
    if playerA_choice == 2 and playerY_choice != 2:
      self.spyA_flag = 1
      #print("Spy will be activated next round for A")
    if playerY_choice == 2 and playerA_choice != 2:
      self.spyY_flag = 1
      #print("Spy will be activated next round for Y")

    #Wizrard rulling | nulify other effects so just plane game check
    if playerA_choice == 5:
      if playerY_choice == 6:
        self.general_bonusY = 0
        #print("General effect for Y DISSMISSED")
      elif playerY_choice == 3:
        self.assasin_flag = 0
        #print("Assasin effect for Y DISSMISSED")
      elif playerY_choice == 4:
        self.holdY -=1
        #print("Extra win point for Y DISSMISSED")
      elif playerY_choice == 2:
        self.spyY_flag = 0
        #print("Spy for Y DISSMISSED")

    if playerY_choice == 5:
      if playerA_choice == 6:
        self.general_bonusA = 0
        #print("General effect for A DISSMISSED")
      elif playerA_choice == 3:
        self.assasin_flag = 0
        #print("Assasin effect for A DISSMISSED")
      elif playerA_choice == 4:
        self.holdA -=1
        #print("Extra win point for A DISSMISSED")
      elif playerA_choice == 2:
        self.spyA_flag = 0
        #print("Spy for A DISSMISSED")

    #MUSICIAN rule

    if playerY_choice == 0:
      if playerA_choice != 5:
        return 0

    if playerA_choice == 0:
      if playerY_choice != 5:
        return 0

    #Prince ruling | win against everyone exept Princes, Musician and another Prince
    if playerA_choice == 7:
      if playerY_choice == 1:  #PRINCESS RULE
        #print("Player Y wins the GAME! PRINCES")
        return 3 # 3 code for GAME WIN player Y
      elif playerY_choice == 7:
        pass
      else:
        #print("Player A wins the round! PRINCE")
        return 1

    if playerY_choice == 7:
      if playerA_choice == 1: #PRINCESS RULE
        #print("Player Y wins the GAME! PRINCES")
        return 4 # 4 code for GAME WIN player A
      elif playerA_choice == 7:
        pass
      else:
        #print("Player Y wins the round! PRINCE")
        return 2

    if self.assasin_flag == 0:
      if playerA_power > playerY_power:
          #print("Player A wins the round!" + " A: ", str(playerA_power) + " > " + " Y: " + str(playerY_power))
          return 1
      elif playerY_power > playerA_power:
          #print("Player Y wins the round!"  + " A: " +  str(playerA_power) + " < " + " Y: " +  str(playerY_power))
          return 2
      else:
          #print("It's a tie!")
          return 0
    elif self.assasin_flag == 1:
      if playerA_power > playerY_power:
          #print("Assasin played: Player Y wins the round!" + " A: ", str(playerA_power) + " > " + " Y: " + str(playerY_power))
          self.assasin_flag = 0
          return 2
      elif playerY_power > playerA_power:
          #print("Assasin played: Player A wins the round!"  + " A: " +  str(playerA_power) + " < " + " Y: " +  str(playerY_power))
          self.assasin_flag = 0
          return 1
      else:
          #print("It's a tie!")
          self.assasin_flag = 0
          return 0

  def get_player_by_name(self, name):
    #print(name, " chose option: ")
    if 'gpt' in name:
      #print(" gpt chosen ")
      return self.get_gpt_choice
    if 'gemini' in name:
      #print(" gemini chosen ")
      return self.get_gpt_choice
    if 'spy' in name:
      #print(" spy chosen ")
      return self.spy_table_choice
    if 'win' in name:
      #print(" win chosen ")
      return self.get_win_strategy
    # random player
    #print(" no option chosen. It is random")
    return self.get_player_choice

  def play_game(self, game_name, player_y_name = 'none', player_a_name ='none'):
    # max_score = 4
    # format (card1, card2, int result)
    game_history  = []

    print("Game Started:", str(game_name))
    error = False
    while (self.playerA_score < self.win_score and self.playerY_score < self.win_score) and self.turn < self.tie_range:
        self.turn = self.turn + 1
        print("TURN: "+str(self.turn))

        player_y = self.get_player_by_name(player_y_name)
        player_a = self.get_player_by_name(player_a_name)

        if self.spyA_flag == 1:
          playerY_choice = player_y(player_y_name, game_history, self.numbers_availableY, None)
          playerA_choice = player_a(player_a_name, game_history, self.numbers_availableA, playerY_choice)
          self.spyA_flag = 0
        elif self.spyY_flag == 1:
          playerA_choice = player_a(player_a_name, game_history, self.numbers_availableA, None)
          playerY_choice = player_y(player_y_name, game_history, self.numbers_availableY, playerA_choice)
          self.spyY_flag = 0
        else:
          playerY_choice = player_y(player_y_name, game_history, self.numbers_availableY, None)
          playerA_choice = player_a(player_a_name, game_history, self.numbers_availableA, None)
        if playerY_choice == -1 or playerA_choice ==-1:
          error= True
          print("ERROR!!! Player has failed to give answer, results of this game should not be considered!")
          break
        else:
          self.numbers_availableY.remove(playerY_choice)
          self.numbers_availableA.remove(playerA_choice)
        winner = self.play_round(playerA_choice, playerY_choice)
        print("TURN: "+str(self.turn)," Player:",player_y_name, " ",format_card(playerY_choice), " Player:", player_a_name, " ", playerA_choice, ": ",format_card(playerA_choice), " Result:", winner)

        round_winner_name = "HOLD"
        if winner == 1:
            if self.holdA > 0:
              self.playerA_score = self.playerA_score + 1 + self.holdA
              self.holdA = 0
              self.holdY = 0
            else:
              round_winner_name = player_y_name
              self.playerA_score = self.playerA_score + 1
        elif winner == 2:
          if self.holdY > 0:
            self.playerY_score = self.playerY_score+ 1 + self.holdY
            self.holdA = 0
            self.holdY = 0
          else:
            round_winner_name = player_a_name
            self.playerY_score = self.playerY_score+ 1
        elif winner == 0:  # Round on hold
            self.holdA = 1
            self.holdY = 1
        elif winner == 3:
          round_winner_name = player_y_name
          self.playerY_score += 10
          # TODO: should we not end all rounds here?
        elif winner == 4:
          round_winner_name = player_a_name
          self.playerA_score += 10
        game_history.append((playerY_choice, playerA_choice, round_winner_name))
        print("TURN: "+str(self.turn), "Scores A:" + str(self.playerA_score)+ " Y:"+ str(self.playerY_score)+" Hold A:"+ str(self.holdA)+" Hold Y:"+ str(self.holdY))

    # Tie
    game_result = 4
    if error:
        game_result= -1
    elif self.playerA_score > self.win_score and self.playerA_score > self.playerY_score:
        #print("Player A wins the game!")
        game_result=1
    elif self.playerY_score > self.win_score and self.playerY_score > self.playerA_score:
        #print("Player Y wins the game!")
        game_result=2

        #ГРА по списку ходів
    print("Game Ended:", game_name, " Result:" , game_result)
    return game_result
    #cards_available.remove(player_choice)


  def play_game_on_list(self, turn_list):

    #print("turn_list IN", turn_list)
    total_pairs = len(turn_list)
    while (self.playerA_score < 4 and self.playerY_score < 4) and self.turn < total_pairs:
        #print("self.turn: ", self.turn, " total_pairs ", total_pairs)
        choice_pair = turn_list[self.turn]
        self.turn = self.turn + 1

        #print("TURN: "+str(self.turn))
        #Spy logic is not needed in this evaluation
        # if self.spyA_flag == 1: #Spy logic is not needed in
        #   playerY_choice = self.get_player_choice('Y', self.numbers_availableY)
        #   playerA_choice = self.spy_table_choice('A', self.numbers_availableA, playerY_choice)
        #   self.spyA_flag = 0
        # elif self.spyY_flag == 1:
        #   playerA_choice = self.get_player_choice('A', self.numbers_availableA)
        #   playerY_choice = self.spy_table_choice('Y', self.numbers_availableY, playerA_choice)
        #   self.spyY_flag = 0
        # else:
        #   playerY_choice = self.get_player_choice('Y', self.numbers_availableY)
        #   playerA_choice = self.get_player_choice('A', self.numbers_availableA)

        playerA_choice = choice_pair[0]
        #print("Player A choice "+str(playerA_choice))
        if playerA_choice in self.numbers_availableA:
          self.numbers_availableA.remove(playerA_choice)
        else:
          print(str(playerA_choice)+" is not in the list: ", self.numbers_availableA)
          return 8

        playerY_choice = choice_pair[1]
        #print("Player Y choice "+str(playerY_choice))
        if playerY_choice in self.numbers_availableY:
          self.numbers_availableY.remove(playerY_choice)
        else:
          print(str(playerY_choice)+" is not in the list: ", self.numbers_availableY)
          return 8


        winner = self.play_round(playerA_choice, playerY_choice)
        #print("winner " +str(winner))
        if winner == 1:
            if self.holdA > 0:
              self.playerA_score = self.playerA_score + 1 + self.holdA
              self.holdA = 0
              self.holdY = 0
            else:
              self.playerA_score = self.playerA_score + 1
        elif winner == 2:
          if self.holdY > 0:
            self.playerY_score = self.playerY_score+ 1 + self.holdY
            self.holdA = 0
            self.holdY = 0
          else:
            self.playerY_score = self.playerY_score+ 1
        elif winner == 0:  # Round on hold
            self.holdA = 1
            self.holdY = 1
        elif winner == 3:
          self.playerY_score += 10
        elif winner == 4:
          self.playerA_score += 10
        #print("Scores A:" + str(self.playerA_score)+ " Y:"+ str(self.playerY_score)+" Hold A:"+ str(self.holdA)+" Hold Y:"+ str(self.holdY))


    if self.playerA_score > 4 and self.playerA_score > self.playerY_score:
        #print("Player A wins the game!")
        return 1
    elif self.playerY_score > 4 and self.playerY_score > self.playerA_score:
        #print("Player Y wins the game!")
        return 2
    elif self.turn < 7:
        #print("Not finished state!")
        return 3
    else:
        #print("tie")
        return 4

**Експерементальна частина**


In [ ]:
# possible player names:
#  "list_player_1" or "list_player_2" - predefined_list must be set.
# "gemini-1.5-flash-latest"
# "gpt-3.5-turbo"
# "gpt-4o"
# "gemini-1.5-pro-latest"
# "win" - for likely win strategy

In [ ]:
rnd_res = {0: 0, 1: 0, 2: 0, 4: 0}

game1 = Game(1)
for iter in range(1000):
  game1.reset(1)
  game_result = game1.play_game(str(iter), 'rnd', 'win')
  if game_result != -1:
    rnd_res[0]+=1
    # TODO: don't count -1, only runs for statistics
    rnd_res[game_result]+=1
    print(rnd_res)

Показано результат, скорочений до останніх рядків (5000).
TURN: 5 Scores A:0 Y:2 Hold A:1 Hold Y:1
TURN: 6
Player: rnd Numbers available: [0, 5] Spy card:  None  Choice:  0
Win strategy : [3, 7, 0, 1, 6, 5, 4] this turn played:  5
TURN: 6  Player: rnd   CARD 0 - musician  Player: win   5 :  CARD 5 - wizard  Result: 1
TURN: 6 Scores A:2 Y:2 Hold A:0 Hold Y:0
TURN: 7
Player: rnd Numbers available: [5] Spy card:  None  Choice:  5
Win strategy : [3, 7, 0, 1, 6, 5, 4] this turn played:  4
TURN: 7  Player: rnd   CARD 5 - wizard  Player: win   4 :  CARD 4 - ambassador  Result: 2
TURN: 7 Scores A:2 Y:3 Hold A:0 Hold Y:0
Game Ended: 815  Result: 4
{0: 816, 1: 169, 2: 131, 4: 516}
Game Started: 816
TURN: 1
Player: rnd Numbers available: [0, 1, 3, 4, 5, 6, 7] Spy card:  None  Choice:  4
Win strategy : [7, 1, 4, 6, 5, 3, 0] this turn played:  7
TURN: 1  Player: rnd   CARD 4 - ambassador  Player: win   7 :  CARD 7 - prince  Result: 1
TURN: 1 Scores A:1 Y:0 Hold A:0 Hold Y:1
TURN: 2
Player: rnd Numb

In [ ]:
rnd_res

{0: 1000, 1: 208, 2: 159, 4: 633}

In [ ]:
gpt_res = {0: 0, 1: 0, 2: 0, 4: 0}

game1 = Game()
for iter in range(100):
  game1.reset()
  game_result = game1.play_game(str(iter), 'gpt-3.5-turbo', 'win')
  if game_result != -1:
    gpt_res[0]+=1
    # TODO: don't count -1, only runs for statistics
    gpt_res[game_result]+=1
    print(gpt_res)

 Win strategy this game:  [7, 3, 0, 1, 4, 5, 6]
 Win strategy this game:  [3, 0, 7, 1, 4, 5, 6]
Game Started: 0
TURN: 1
Player: gpt-3.5-turbo Request: Your ONLY Available cards left:CARD 0 - musician,CARD 1 - princess,CARD 3 - assassin,CARD 4 - ambassador,CARD 5 - wizard,CARD 6 - general,CARD 7 - prince
Your turn to give the best strategic card from one of the available. Remember to answer only as card strength/number
gpt-3.5-turbo  answer: 1 .  CARD 1 - princess
Win strategy : [3, 0, 7, 1, 4, 5, 6] this turn played:  3
TURN: 1  Player: gpt-3.5-turbo   CARD 1 - princess  Player: win   3 :  CARD 3 - assassin  Result: 2
TURN: 1 Scores A:0 Y:1 Hold A:0 Hold Y:0
TURN: 2
Player: gpt-3.5-turbo Request: GAME HISTORY: 
ROUND 1, YOUR's CARD 1 - princess, OPPONENT's CARD 3 - assassin. OPPONENT WIN
Your ONLY Available cards left:CARD 0 - musician,CARD 3 - assassin,CARD 4 - ambassador,CARD 5 - wizard,CARD 6 - general,CARD 7 - prince
Your turn to give the best strategic card from one of the availab

In [ ]:
gpt_res

{0: 100, 1: 11, 2: 48, 4: 41}

In [ ]:
gpt4_res = {0: 0, 1: 0, 2: 0, 4: 0}

game1 = Game()
for iter in range(100):
  game1.reset()
  game_result = game1.play_game(str(iter), 'gpt-4-turbo', 'win')
  if game_result != -1:
    gpt4_res[0]+=1
    # TODO: don't count -1, only runs for statistics
    gpt4_res[game_result]+=1
    print(gpt4_res)

Показано результат, скорочений до останніх рядків (5000).
ROUND 2, YOUR's CARD 0 - musician, OPPONENT's CARD 0 - musician. HOLD
ROUND 3, YOUR's CARD 6 - general, OPPONENT's CARD 3 - assassin. HOLD
ROUND 4, YOUR's CARD 4 - ambassador, OPPONENT's CARD 1 - princess. HOLD
ROUND 5, YOUR's CARD 5 - wizard, OPPONENT's CARD 5 - wizard. HOLD
Your ONLY Available cards left:CARD 1 - princess,CARD 3 - assassin
Your turn to give the best strategic card from one of the available. Remember to answer only as card strength/number
gpt-4-turbo  answer: 3 .  CARD 3 - assassin
Win strategy : [7, 0, 3, 1, 5, 4, 6] this turn played:  4
TURN: 6  Player: gpt-4-turbo   CARD 3 - assassin  Player: win   4 :  CARD 4 - ambassador  Result: 2
TURN: 6 Scores A:2 Y:4 Hold A:0 Hold Y:0
Game Ended: 16  Result: 4
{0: 17, 1: 6, 2: 1, 4: 10}
 Win strategy this game:  [3, 0, 7, 1, 4, 5, 6]
Game Started: 17
TURN: 1
Player: gpt-4-turbo Request: Your ONLY Available cards left:CARD 0 - musician,CARD 1 - princess,CARD 3 - assassi

In [ ]:
gpt4_res

{0: 100, 1: 29, 2: 3, 4: 68}

In [ ]:
# 0 for number of successful runs
gemini_res = {0: 0, 1: 0, 2: 0, 4: 0}

game1 = Game()

for iter in range(200):
  game1.reset()
  game_result = game1.play_game(str(iter), 'win', 'gemini-1.5-flash-latest')
  if game_result != -1:
    gemini_res[0]+=1
    # TODO: don't count -1, only runs for statistics
    gemini_res[game_result]+=1
    print(gemini_res)
# gpt-3.5-turbo {1: 29, 2: 15, 4: 56}
# gpt-3.5-turbo {0: 100, 1: 23, 2: 13, 4: 64}
# gemini-1.5-flash-latest" {0: 200, 1: 38, 2: 52, 4: 110}
# gpt-4o {0: 199, 1: 49, 2: 36, 4: 114}
# gemini-1.5-pro-latest {0: 199, 1: 39, 2: 45, 4: 115}
# print(gpt_res)
# print("Ended. results:", )
# raise Exception("stop")
#
#game1 = Game()
#game1.play_game()

# rnd_res = {1: 0, 2: 0, 4: 0}
# game1 = Game()
# for x in range(10000):
#  game1.reset()
#  rnd_res[game1.play_game('win_strategy')]+=1

# print(rnd_res)

Показано результат, скорочений до останніх рядків (5000).
ROUND 2, YOUR's CARD 1 - princess, OPPONENT's CARD 0 - musician. HOLD
ROUND 3, YOUR's CARD 4 - ambassador, OPPONENT's CARD 6 - general. HOLD
Your ONLY Available cards left:CARD 1 - princess,CARD 3 - assassin,CARD 4 - ambassador,CARD 5 - wizard
Your turn to give the best strategic card from one of the available. Remember to answer only as card strength/number
gemini-1.5-flash-latest  answer: 3 .  CARD 3 - assassin
TURN: 4  Player: win   CARD 6 - general  Player: gemini-1.5-flash-latest   3 :  CARD 3 - assassin  Result: 1
TURN: 4 Scores A:3 Y:0 Hold A:0 Hold Y:0
TURN: 5
Win strategy : [7, 1, 4, 6, 5, 3, 0] this turn played:  5
Player: gemini-1.5-flash-latest Request: GAME HISTORY: 
ROUND 1, YOUR's CARD 7 - prince, OPPONENT's CARD 7 - prince. HOLD
ROUND 2, YOUR's CARD 1 - princess, OPPONENT's CARD 0 - musician. HOLD
ROUND 3, YOUR's CARD 4 - ambassador, OPPONENT's CARD 6 - general. HOLD
ROUND 4, YOUR's CARD 6 - general, OPPONENT's C

In [ ]:
gemini_res

{0: 200, 1: 30, 2: 26, 4: 144}

**Game on 6**

In [ ]:
gpt_res_6 = {0: 0, 1: 0, 2: 0, 4: 0}

game6 = Game(2)
for iter in range(100):
  game6.reset(2)
  game_result = game6.play_game(str(iter), 'gpt-3.5-turbo', 'win')
  if game_result != -1:
    gpt_res_6[0]+=1
    # TODO: don't count -1, only runs for statistics
    gpt_res_6[game_result]+=1
    print(gpt_res_6)

Game Started: 0
TURN: 1
Player: gpt-3.5-turbo Request: Your ONLY Available cards left:CARD 0 - musician,CARD 1 - princess,CARD 3 - assassin,CARD 5 - wizard,CARD 6 - general,CARD 7 - prince
Your turn to give the best strategic card from one of the available. Remember to answer only as card strength/number
gpt-3.5-turbo  answer: 1 .  CARD 1 - princess
Win strategy : [7, 3, 0, 6, 5, 1] this turn played:  7
TURN: 1  Player: gpt-3.5-turbo   CARD 1 - princess  Player: win   7 :  CARD 7 - prince  Result: 3
TURN: 1 Scores A:0 Y:10 Hold A:0 Hold Y:0
Game Ended: 0  Result: 2
{0: 1, 1: 0, 2: 1, 4: 0}
Game Started: 1
TURN: 1
Player: gpt-3.5-turbo Request: Your ONLY Available cards left:CARD 0 - musician,CARD 1 - princess,CARD 3 - assassin,CARD 5 - wizard,CARD 6 - general,CARD 7 - prince
Your turn to give the best strategic card from one of the available. Remember to answer only as card strength/number
gpt-3.5-turbo  answer: 1 .  CARD 1 - princess
Win strategy : [7, 0, 1, 6, 5, 3] this turn played:

In [ ]:
print(gpt_res_6)

{0: 100, 1: 0, 2: 94, 4: 6}


In [ ]:
gemini_res_6 = {0: 0, 1: 0, 2: 0, 4: 0}

game6 = Game(2)
for iter in range(100):
  game6.reset(2)
  game_result = game6.play_game(str(iter), 'gemini-1.5-flash-latest', 'win')
  if game_result != -1:
    gemini_res_6[0]+=1
    # TODO: don't count -1, only runs for statistics
    gemini_res_6[game_result]+=1
    print(gemini_res_6)

**Game on 5 cards**

In [ ]:
gpt_res_5 = {0: 0, 1: 0, 2: 0, 4: 0}

game5 = Game(3)
for iter in range(100):
  game5.reset(3)
  game_result = game5.play_game(str(iter), 'gpt-3.5-turbo', 'win')
  if game_result != -1:
    gpt_res_5[0]+=1
    # TODO: don't count -1, only runs for statistics
    gpt_res_5[game_result]+=1
    print(gpt_res_5)

Game Started: 0
TURN: 1
Player: gpt-3.5-turbo Request: Your ONLY Available cards left:CARD 0 - musician,CARD 1 - princess,CARD 3 - assassin,CARD 5 - wizard,CARD 7 - prince
Your turn to give the best strategic card from one of the available. Remember to answer only as card strength/number
gpt-3.5-turbo  answer: 1 .  CARD 1 - princess
Win strategy : [7, 0, 5, 3, 1] this turn played:  7
TURN: 1  Player: gpt-3.5-turbo   CARD 1 - princess  Player: win   7 :  CARD 7 - prince  Result: 3
TURN: 1 Scores A:0 Y:10 Hold A:0 Hold Y:0
Game Ended: 0  Result: 2
{0: 1, 1: 0, 2: 1, 4: 0}
Game Started: 1
TURN: 1
Player: gpt-3.5-turbo Request: Your ONLY Available cards left:CARD 0 - musician,CARD 1 - princess,CARD 3 - assassin,CARD 5 - wizard,CARD 7 - prince
Your turn to give the best strategic card from one of the available. Remember to answer only as card strength/number
gpt-3.5-turbo  answer: 1 .  CARD 1 - princess
Win strategy : [7, 0, 5, 3, 1] this turn played:  7
TURN: 1  Player: gpt-3.5-turbo   CAR

In [ ]:
print(gpt_res_5)

{0: 100, 1: 0, 2: 98, 4: 2}


In [ ]:
gemini_res_5 = {0: 0, 1: 0, 2: 0, 4: 0}

game5 = Game(3)
for iter in range(100):
  game5.reset(3)
  game_result = game5.play_game(str(iter), 'gemini-1.5-flash-latest', 'win')
  if game_result != -1:
    gemini_res_5[0]+=1
    # TODO: don't count -1, only runs for statistics
    gemini_res_5[game_result]+=1
    print(gemini_res_5)

In [ ]:
gemini_res_5

**Генерація дерева рішень**


In [ ]:
from itertools import permutations

In [ ]:
#Приклад гри по лісту пар
tuple_list = [(0,0), (1, 1), (3,3), (4,4), (5,5), (6,6), (7,7)]
#tuple_list = [(0,0), (1, 1), (2, 2)]
game1 = Game()
game1.reset()
value = game1.play_game_on_list(tuple_list)
print("VALUE "+str(value))

VALUE 3


In [ ]:
def compare_tuples(win_comb, first_elements):
    if len(first_elements) <= len(win_comb):
        return first_elements == win_comb[:len(first_elements)]
    else:
        return first_elements[:len(win_comb)] == win_comb and len(first_elements) == len(win_comb) + 1


In [ ]:
class Game_Solution:



  def __init__(self, card_limit):
    self.dict_states = {}
    self.combination_stat_composed = {}
    self.numbers_available = list(range(8))
    if card_limit > 0:
      self.limit_game(card_limit)

    tuple_list = []

    for A in self.numbers_available:
          for Y in self.numbers_available:
            tuple_list.append((A, Y))

    self.stack = []

    for element in tuple_list:
      temp_list = []
      temp_list.append(element)
      #print(temp_list)
      self.stack.append(temp_list)

  def limit_game(self, cards_limit):
    if cards_limit > 0:
      self.numbers_available.remove(2) #Прибираємо Шпигуна
    if cards_limit > 1:
      self.numbers_available.remove(4) #Прибираємо Амбасадора
    if cards_limit > 2:
      self.numbers_available.remove(6) #Прибираємо Генерала
    if cards_limit > 3:
      self.numbers_available.remove(5) #Прибираємо Мага
    if cards_limit > 4:
      self.numbers_available.remove(0) #Прибираємо Музикант

  def generate_dict(self, card_limit):

    game_ = Game(card_limit)
    while(len(self.stack)!=0):
      curr = self.stack.pop()
      game_.reset(card_limit)
      #print("Curr IN", curr)
      result = game_.play_game_on_list(curr)

      if result == 3:
        #perm
        listA = self.numbers_available.copy()
        listY = self.numbers_available.copy()
        for pair in curr:
          listA.remove(pair[0])
          #print("A ", listA)
          listY.remove(pair[1])
          #print("Y ", listY)
        for A in listA:
          for Y in listY:
            new_pair = curr.copy()
            new_pair.append((A, Y))
            #print("new element", new_pair[3])
            self.stack.append(new_pair)
            #print(" ", new_pair )
      else:
        q = tuple(curr)
        self.dict_states[q]=result
      #print("Curr OUT", curr)

  def win_strategies(self):
    combination_stat = {}
    perm = permutations(self.numbers_available)
    win_combinations = [tpl for tpl in perm]
    print(win_combinations)

    for key, value in self.dict_states.items():
      fist_in_key = tuple([pair[0] for pair in key])
      if fist_in_key in combination_stat.keys():
        pass
      else:
        combination_stat[fist_in_key] = [0, 0, 0, 0]

    for combination in win_combinations:
      fist_in_key = tuple(combination)
      self.combination_stat_composed[fist_in_key] = [0, 0, 0, 0]

    for key, value in self.dict_states.items():
      fist_in_key = tuple([pair[0] for pair in key])
      #
      if value == 4: # tie
        combination_stat[fist_in_key][2] +=1
        #ties1 +=1
      elif value == 2: #lose
        combination_stat[fist_in_key][1] +=1
        #loses1 +=1
      elif value == 1: #win
        combination_stat[fist_in_key][0] +=1
        #wins1 +=1
      else:
          print("error")

    for key, value in self.combination_stat_composed.items():
      for i in range(1, len(key) + 1):
        # Create a slice of the tuple from the beginning to the current length
        sub_tuple = key[:i]
        # Call the function with the current slice
        if sub_tuple in combination_stat.keys():
          value[0] += combination_stat[sub_tuple][0]
          value[1] += combination_stat[sub_tuple][1]
          value[2] += combination_stat[sub_tuple][2]
    for key, value in self.combination_stat_composed.items():
      value[3] = (value[0] + value[2])/(value[0] + value[1] + value[2])
      if value[3] >0.8:
        print(key, " ", value)

  def show_win_strategies(self, percentage):
    for key, value in self.combination_stat_composed.items():
      if value[3] > percentage:
        print(key, " ", value)

In [ ]:
tree_minus_two = Game_Solution(2)
tree_minus_two.generate_dict(2)
tree_minus_two.win_strategies()

In [ ]:
tree_minus_two.show_win_strategies(0.95)

(7, 0, 1, 6, 5, 3)   [41, 1, 0, 0.9761904761904762]
(7, 0, 3, 6, 5, 1)   [78, 2, 0, 0.975]
(7, 0, 6, 5, 3, 1)   [82, 3, 0, 0.9647058823529412]
(7, 3, 0, 6, 5, 1)   [63, 2, 0, 0.9692307692307692]
(7, 6, 0, 5, 3, 1)   [85, 2, 0, 0.9770114942528736]
(7, 6, 5, 0, 3, 1)   [121, 5, 0, 0.9603174603174603]


In [ ]:
tree_minus_3 = Game_Solution(3)
tree_minus_3.generate_dict(3)

In [ ]:
tree_minus_3.win_strategies()

[(0, 1, 3, 5, 7), (0, 1, 3, 7, 5), (0, 1, 5, 3, 7), (0, 1, 5, 7, 3), (0, 1, 7, 3, 5), (0, 1, 7, 5, 3), (0, 3, 1, 5, 7), (0, 3, 1, 7, 5), (0, 3, 5, 1, 7), (0, 3, 5, 7, 1), (0, 3, 7, 1, 5), (0, 3, 7, 5, 1), (0, 5, 1, 3, 7), (0, 5, 1, 7, 3), (0, 5, 3, 1, 7), (0, 5, 3, 7, 1), (0, 5, 7, 1, 3), (0, 5, 7, 3, 1), (0, 7, 1, 3, 5), (0, 7, 1, 5, 3), (0, 7, 3, 1, 5), (0, 7, 3, 5, 1), (0, 7, 5, 1, 3), (0, 7, 5, 3, 1), (1, 0, 3, 5, 7), (1, 0, 3, 7, 5), (1, 0, 5, 3, 7), (1, 0, 5, 7, 3), (1, 0, 7, 3, 5), (1, 0, 7, 5, 3), (1, 3, 0, 5, 7), (1, 3, 0, 7, 5), (1, 3, 5, 0, 7), (1, 3, 5, 7, 0), (1, 3, 7, 0, 5), (1, 3, 7, 5, 0), (1, 5, 0, 3, 7), (1, 5, 0, 7, 3), (1, 5, 3, 0, 7), (1, 5, 3, 7, 0), (1, 5, 7, 0, 3), (1, 5, 7, 3, 0), (1, 7, 0, 3, 5), (1, 7, 0, 5, 3), (1, 7, 3, 0, 5), (1, 7, 3, 5, 0), (1, 7, 5, 0, 3), (1, 7, 5, 3, 0), (3, 0, 1, 5, 7), (3, 0, 1, 7, 5), (3, 0, 5, 1, 7), (3, 0, 5, 7, 1), (3, 0, 7, 1, 5), (3, 0, 7, 5, 1), (3, 1, 0, 5, 7), (3, 1, 0, 7, 5), (3, 1, 5, 0, 7), (3, 1, 5, 7, 0), (3, 1, 7, 0, 

In [ ]:
tree_minus_3.show_win_strategies(0.9)

(7, 0, 3, 1, 5)   [18, 1, 0, 0.9473684210526315]
(7, 0, 3, 5, 1)   [18, 1, 0, 0.9473684210526315]
(7, 0, 5, 1, 3)   [18, 1, 0, 0.9473684210526315]
(7, 0, 5, 3, 1)   [19, 1, 0, 0.95]
(7, 3, 0, 1, 5)   [18, 1, 0, 0.9473684210526315]
(7, 3, 0, 5, 1)   [16, 1, 0, 0.9411764705882353]
(7, 3, 5, 0, 1)   [18, 1, 0, 0.9473684210526315]
(7, 3, 5, 1, 0)   [18, 1, 0, 0.9473684210526315]
(7, 5, 0, 1, 3)   [18, 1, 0, 0.9473684210526315]
(7, 5, 0, 3, 1)   [18, 1, 0, 0.9473684210526315]
(7, 5, 1, 3, 0)   [10, 1, 0, 0.9090909090909091]
(7, 5, 3, 0, 1)   [18, 1, 0, 0.9473684210526315]
(7, 5, 3, 1, 0)   [18, 1, 0, 0.9473684210526315]


In [ ]:
numbers_available = list(range(8))
numbers_available.remove(2) #Прибираємо шпигуна
tuple_list = []
for A in numbers_available:
      for Y in numbers_available:
        tuple_list.append((A, Y))

In [ ]:
stack = []
for element in tuple_list:
  temp_list = []
  temp_list.append(element)
  #print(temp_list)
  stack.append(temp_list)

In [ ]:
game_ = Game()
dict_states = {}
while(len(stack)!=0):
  curr = stack.pop()
  game_.reset()
  #print("Curr IN", curr)
  result = game_.play_game_on_list(curr)

  if result == 3:
    #perm
    listA = numbers_available.copy()
    listY = numbers_available.copy()
    for pair in curr:
      listA.remove(pair[0])
      #print("A ", listA)
      listY.remove(pair[1])
      #print("Y ", listY)
    for A in listA:
      for Y in listY:
        new_pair = curr.copy()
        new_pair.append((A, Y))
        #print("new element", new_pair[3])
        stack.append(new_pair)
        #print(" ", new_pair )
  else:
    q = tuple(curr)
    dict_states[q]=result
  #print("Curr OUT", curr)


In [ ]:
from google.colab import drive
import csv
from csv import DictReader
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#print(dict_states)
#print(dict_states.keys())

with open('/content/drive/My Drive/States/state_no_spy.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    for key, value in dict_states.items():
        # Write each key-value pair to the CSV
        writer.writerow([key, value])
    print('Done writing dict to a csv file')


Done writing dict to a csv file


In [ ]:
dict_states[((0, 0), (1, 3), (7, 1))]

{((7, 7), (6, 6), (5, 5), (4, 4), (3, 3), (1, 1), (0, 0)): 4,
 ((7, 7), (6, 6), (5, 5), (4, 4), (3, 3), (1, 0), (0, 1)): 4,
 ((7, 7), (6, 6), (5, 5), (4, 4), (3, 3), (0, 1), (1, 0)): 4,
 ((7, 7), (6, 6), (5, 5), (4, 4), (3, 3), (0, 0), (1, 1)): 4,
 ((7, 7), (6, 6), (5, 5), (4, 4), (3, 1), (1, 3), (0, 0)): 4,
 ((7, 7), (6, 6), (5, 5), (4, 4), (3, 1), (1, 0), (0, 3)): 4,
 ((7, 7), (6, 6), (5, 5), (4, 4), (3, 1), (0, 3), (1, 0)): 4,
 ((7, 7), (6, 6), (5, 5), (4, 4), (3, 1), (0, 0), (1, 3)): 4,
 ((7, 7), (6, 6), (5, 5), (4, 4), (3, 0), (1, 3), (0, 1)): 4,
 ((7, 7), (6, 6), (5, 5), (4, 4), (3, 0), (1, 1), (0, 3)): 4,
 ((7, 7), (6, 6), (5, 5), (4, 4), (3, 0), (0, 3), (1, 1)): 4,
 ((7, 7), (6, 6), (5, 5), (4, 4), (3, 0), (0, 1), (1, 3)): 4,
 ((7, 7), (6, 6), (5, 5), (4, 4), (1, 3), (3, 1), (0, 0)): 4,
 ((7, 7), (6, 6), (5, 5), (4, 4), (1, 3), (3, 0), (0, 1)): 4,
 ((7, 7), (6, 6), (5, 5), (4, 4), (1, 3), (0, 1), (3, 0)): 4,
 ((7, 7), (6, 6), (5, 5), (4, 4), (1, 3), (0, 0), (3, 1)): 4,
 ((7, 7)

In [ ]:
import ast
# opening the file using "with"
# statement
new_dict = {}

with open("/content/drive/My Drive/States/state_00.csv", 'r') as file:
   # Convert the string representation of the tuple back to a tuple using ast.literal_eval
    reader = csv.reader(file)
    for row in reader:
        # Convert the string representation of the tuple back to a tuple using ast.literal_eval
        key = ast.literal_eval(row[0])
        value = int(row[1])
        new_dict[key] = value

In [ ]:
def compare_tuples(win_comb, first_elements):
    if len(first_elements) <= len(win_comb):
        return first_elements == win_comb[:len(first_elements)]
    else:
        return first_elements[:len(win_comb)] == win_comb and len(first_elements) == len(win_comb) + 1

In [ ]:
from itertools import permutations
perm = permutations([0, 1, 3, 4, 5, 6, 7])
win_combinations = [tpl for tpl in perm]
combination_stat = {}

In [ ]:
type(win_combinations)

list

In [ ]:
for key, value in dict_states.items():
  fist_in_key = tuple([pair[0] for pair in key])
  if fist_in_key in combination_stat.keys():
    pass
  else:
    combination_stat[fist_in_key] = [0, 0, 0, 0]

In [ ]:
combination_stat_composed = {}
for combination in win_combinations:
  fist_in_key = tuple(combination)
  combination_stat_composed[fist_in_key] = [0, 0, 0, 0]

In [ ]:
for key, value in dict_states.items():
  fist_in_key = tuple([pair[0] for pair in key])
  #
  if value == 4: # tie
    combination_stat[fist_in_key][2] +=1
    #ties1 +=1
  elif value == 2: #lose
    combination_stat[fist_in_key][1] +=1
    #loses1 +=1
  elif value == 1: #win
    combination_stat[fist_in_key][0] +=1
    #wins1 +=1
  else:
      print("error")
  # tw_part = (wins1+ties1)/(wins1+loses1+ties1)
  # combination_stat[combination] = (wins1, loses1, ties1, tw_part)

In [ ]:
for key, value in combination_stat_composed.items():
  for i in range(1, len(key) + 1):
    # Create a slice of the tuple from the beginning to the current length
    sub_tuple = key[:i]
    # Call the function with the current slice
    if sub_tuple in combination_stat.keys():
      value[0] += combination_stat[sub_tuple][0]
      value[1] += combination_stat[sub_tuple][1]
      value[2] += combination_stat[sub_tuple][2]

In [ ]:
for key, value in combination_stat_composed.items():
  #value[3] = (value[0] + value[2])/(value[0] + value[1] + value[2])
  if value[3] >0.9540:
    print(key, " ", value)

(0, 7, 1, 4, 6, 5, 3)   [355, 61, 995, 0.9567682494684621]
(3, 0, 7, 1, 4, 5, 6)   [717, 87, 1293, 0.9585121602288984]
(3, 0, 7, 1, 6, 5, 4)   [538, 60, 1084, 0.9643281807372176]
(3, 7, 0, 1, 4, 5, 6)   [518, 79, 1363, 0.9596938775510204]
(3, 7, 0, 1, 6, 5, 4)   [419, 55, 1186, 0.9668674698795181]
(7, 0, 1, 4, 6, 5, 3)   [402, 56, 1184, 0.9658952496954933]
(7, 0, 3, 1, 5, 4, 6)   [723, 93, 1341, 0.9568845618915159]
(7, 1, 4, 6, 5, 3, 0)   [44, 51, 1021, 0.9543010752688172]
(7, 3, 0, 1, 4, 5, 6)   [555, 77, 1148, 0.9567415730337079]
(7, 3, 0, 1, 6, 5, 4)   [407, 61, 1082, 0.9606451612903226]


In [ ]:
print(gpt_res)

{1: 15, 2: 22, 4: 63}


In [ ]:
gpt_res35 = {1: 0, 2: 0, 4: 0}
game1 = Game()
for iter in range(100):
  game1.reset()
  gpt_res35[game1.play_game_on_random('gpt')]+=1

TURN: 1
Numbers available: [0, 1, 2, 3, 4, 5, 6, 7]
Player Y's choise for turn:0
Options are:  0, 1, 2, 3, 4, 5, 6, 7,
ChatCompletionMessage(content='Chosen card: 5', role='assistant', function_call=None, tool_calls=None)
[0, 1, 2, 3, 4, 5, 6, 7]  number:  5
Scores A:1 Y:0 Hold A:0 Hold Y:0
TURN: 2
Numbers available: [1, 2, 3, 4, 5, 6, 7]
Player Y's choise for turn:2
Options are:  0, 1, 2, 3, 4, 6, 7,
ChatCompletionMessage(content='4', role='assistant', function_call=None, tool_calls=None)
[0, 1, 2, 3, 4, 6, 7]  number:  4
Scores A:3 Y:0 Hold A:0 Hold Y:0
TURN: 3
Options are:  0, 1, 2, 3, 6, 7,
ChatCompletionMessage(content='4', role='assistant', function_call=None, tool_calls=None)
[0, 1, 2, 3, 6, 7]  number:  4
Options are:  0, 1, 2, 3, 6, 7,
ChatCompletionMessage(content='I choose 7.', role='assistant', function_call=None, tool_calls=None)
[0, 1, 2, 3, 6, 7]  number:  7
Spy choise!
Numbers available: [1, 3, 4, 5, 6, 7]
Option list
['Y', 'Y', 'Y', 'H', 'Y', 'A', 'A', 'H']
Spy predict